In [8]:
import os
from bosonnlp import BosonNLP
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import  LatentDirichletAllocation

In [9]:
news_dir = '/home/ewan/PycharmProjects/news_spider/news/'
file_list = os.listdir(news_dir)

In [9]:
print file_list[0]

网络直播平台现刷单返现广告 男子被骗5千余元


In [10]:
nlp = BosonNLP('FuHSE7Vf.13924.jadflTdrQLWx')

In [20]:
splitted_titles = []

SIZE = len(file_list)
count = 0

stop_tags = ['m', 'w', 't', 'q', 'u', 'k', 'h', 'o', 'y', 'c', 'p', 'd', 'r']

for title in file_list:
    count += 1
    if count % (SIZE / 100) == 0:
        print count
    result = nlp.tag(title)
    words = ''
    for index, word in enumerate(result[0]['word']):
        stop = False
        for tag in stop_tags:
            if tag in result[0]['tag'][index]:
                stop = True
                break
        if stop is False:
            words += word + ' '

    splitted_titles.append(words)

43


86


129


172


215


...

4128


4171


4214


4257


4300


In [12]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print 'Topic #%d:' % topic_idx
        print ' '.join([feature_names[i]
                        for i in topic.argsort()[: -n_top_words - 1 : -1]])
        print

In [39]:
# Use tf (raw term count) features for LDA
print("Extracting tf feature for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=500)

tf = tf_vectorizer.fit_transform(splitted_titles)
# print tf_vectorizer.get_feature_names()[456]

print 'Fitting LDA models with tf features...'
lda = LatentDirichletAllocation(n_topics=5, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)

Extracting tf feature for LDA...
Fitting LDA models with tf features...


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=5, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [40]:
print '\nTopic in LDA model:'
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)


Topic in LDA model:
Topic #0:
中国 回应 学生 问题 发现 国家 上海 大选 官员 嫌犯 警方 荷兰 电缆 妻子 食品 印度 世界 局长 影响 老师

Topic #1:
韩国 老人 媒体 手机 朝鲜 山东 限购 云南 法院 孩子 广州 女孩 刑拘 检方 济南 调查 取消 举行 国务卿 机构

Topic #2:
女子 日本 北京 中国 总统 特朗普 外媒 网友 共享 小时 诈骗 单车 专家 台湾 游客 大学生 楼市 网络 广东 总理

Topic #3:
男子 调查 人员 河南 公司 城市 企业 交警 报告 江苏 书记 抢劫 女儿 接受 萨德 女友 涉嫌 官方 法国 生活

Topic #4:
美国 英国 死亡 司机 政府 发生 警方 警察 事件 安全 儿子 全国 受伤 要求 医院 医生 袭击 曝光 事故 小伙



In [44]:
# Transfrom to topic probabilities
topic_prob = np.array(lda.transform(tf))
news_topics = []
for probs in topic_prob:
    news_topics.append(np.where(probs >= 0.4)[0].tolist())

topics = {}
for topic in xrange(5):
    topics[topic] = []

for index, new_topics in enumerate(news_topics):
    for topic in new_topics:
        topics[topic].append(index)

for new_id in topics[0][:30]:
    print file_list[new_id]

印媒:中国防长访斯里兰卡和尼泊尔 印度密切关注
鼓浪屿、可可西里申遗 中国或成世界遗产总数第一
神奇!老师把试卷放大一倍 成绩中下学生考出高分
内陆核电站建设何时破冰?国家能源局:暂无时间表
越南警方发现五只冰冻老虎尸体 器官遗失
山东前2月检出不合格进口产品9批 主要来自韩国
八旬老两口离婚37年后妻子患癌 丈夫:复婚
4名嫌疑人胁迫强奸少女拍视频 3人为未成年人
成都地铁承认使用问题品牌电缆 重庆地铁:未使用
俄媒：泰方称中国曾提议以2艘价格出售3艘潜艇
老外品尝中国普洱茶:味道像"腐烂的堆肥"
开发商进小区"圈地"称旧改签约奖5万送车位使用权
台北地检署起诉马英九教唆泄密 马全盘否认犯罪
菲媒炒作中国科考船过航"宾汉隆起" 外交部回应
乐天关闭中国境内9成门店 供应商赴北京总部催款
文在寅:望代总统公正对待大选 曾称对方不应参选
内蒙古村支书66刀捅死钉子户案二审结束 择日宣判
不会被宰还享受按摩 荷兰一农场成“猪的乐土”
上海一名小学生午餐时意外身亡 医生疑窒息所致
初中生疑因被索要钱财未给 被多名学生殴打身亡
气象专家:人工干预祁连山天气不影响周边大气候
中方回应日"加贺号"服役:应汲取被击沉的历史教训
18岁外卖小哥用英文发送餐短信:欲自学英语上大学
夫妻离婚父亲藏儿3年 男孩拒认生母称妈妈坏人
北京轨交建设公司:工程均未采用陕西奥凯电缆产品
的哥借钱给乘客救急 乘客迟迟不还被起诉
海外投资之痛:毛阿敏丈夫近10亿美元投资或打水漂
成都:去年以来市纪检监察机关立案3583件
外交部正核实"一中国支教教师在缅北冲突中遇难"
农大“学霸”就餐发现足丝蚁致信校长走红 
